In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as  sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Read data in**

In [ ]:

df = pd.read_csv('../input/vietnamesejobs/jobdata.csv')
searching = pd.read_csv('../input/users-behavior-data/ls_tim_kiem.csv')
reading = pd.read_csv('../input/users-behavior-data/ls_xem_tin.csv')

Thống kê số lượt xem của mỗi tin


In [ ]:
print(reading.columns)
#sns.countplot(x ='id tin ',data = reading )

In [ ]:
count = reading.groupby(by = 'id tin ',as_index = False)['id ứng viên '].count()
count = count.rename(columns = {'id ứng viên ':'luot xem '})
count[:10]

Lượt xem trung bình mỗi tin là 1.42, khá là thấp

In [ ]:
print(count['luot xem '].min())
print(count['luot xem '].mean())

In [ ]:
readingCount = pd.merge(reading,count,on = 'id tin ')
readingCount.head(15)

Thống kê số lượng tin mỗi ứng viên đã đọc

In [ ]:
readingCount['id ứng viên '].value_counts()

In [ ]:
stt = [519,594,599,602]
for i in stt:
    print(reading[reading['STT ']== i])

Gom nhóm các ngành nghề mà mỗi ứng viên quan tâm

In [ ]:
searching['Tên ngành nghề '] = searching['Tên ngành nghề '].apply(lambda s:s +',')

user_df = searching.groupby(by = 'id ứng viên ',as_index = True)['Tên ngành nghề '].sum()
user_df = user_df.apply(lambda s:s.strip()[:-1])

user_df1 = user_df.apply(lambda s:s.split(','))
print(user_df1)

print(len(user_df1))


Mỗi ứng viên có thể quan tâm đến 1 hoặc nhiều ngành nghề => bài toán Multilable

Dùng MultiLabelBinarizer để đưa dữ liệu về ma trận thưa thớt, khi đó mỗi sample được biểu diễn bằng 1 vector trong không gian 286 chiều, phục vụ cho việc tính cosine_similarity ( tính khoảng cách giữa các vector trong siêu không gian)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
user_df = mlb.fit_transform(user_df1)
#print(user_df[:10])
from sklearn.metrics.pairwise import cosine_similarity
user_sim = cosine_similarity(user_df)
user_sim.shape

Tương quan giữa ứng viên và tin 
Khi ứng viên xem 1 tin thì (id tin,id ứng viên) = 1

In [ ]:
users_news = pd.pivot_table(reading,index = 'id ứng viên ',columns = 'id tin ')
users_news = users_news.fillna(0)
users_news

In [ ]:
users_users= cosine_similarity(users_news)
users_users.shape

In [ ]:
# most_sim_user_sorted = []
# compared_user = list(enumerate(user_sim[user_index]))
# sorted_user_list = sorted(compared_user, key=lambda x:x[1], reverse=True)
# for usr in sorted_user_list:
#     usr_id = user_df1['id ứng viên '][usr[0]]
#     print(usr_id)
#     if ( usr_id in similar_users):
#         most_sim_user_sorted.append(usr)

Nối 2 bảng để lọc ra các thông tin 'id tin ' khớp nhau

In [ ]:
join_list = pd.merge(readingCount,df,left_on = 'id tin ',right_on = 'Id tin ')
#print(join_list.columns)
print('Available user: ',join_list['id ứng viên '].unique())
print(len(join_list['id ứng viên '].unique()))

* Input là id ứng viên cần gợi ý và id tin mà ứng viên đó đang xem 
* Sau đó tìm ra các ứng viên khác cũng đang quan tâm tới tin đó
* Tìm ra các tin mà ứng viên khác cũng đọc

In [ ]:
#666464
#665125
user_id = 651551          
current_news_id = join_list[join_list['id ứng viên ']==user_id]['id tin '].values[0]
print('current_news_id: ',(current_news_id))
user_index = join_list[join_list['id ứng viên ']==user_id].index.tolist()[0]
print('user_index :',user_index)

similar_users=join_list[join_list['Id tin ']==current_news_id]['id ứng viên ']


# users_list = list(enumerate(users_users[user_index]))
# #print('users_list',users_list[:10])
# sorted_list = sorted(users_list,key=lambda x:x[1], reverse=True) 
# print("Current users: ",join_list['id ứng viên '][user_index])
# print("Similar users: \n")
# similar_users = []
# sorted_list = list(set(sorted_list))
# for usr in sorted_list[1:6]:
#     similar_users.append(reading['id ứng viên '][usr[0]])
print('similar_users: ',similar_users.values[0])

news_list = []
for user in similar_users:
    news = join_list[join_list['id ứng viên ']==user]['id tin '].values
    for nw in news:
        news_list.append(nw)

In [ ]:
print(list(set(news_list)))

In [ ]:
print(df.columns)
print(readingCount.columns)
join_list = pd.merge(readingCount,df,left_on = 'id tin ',right_on = 'Id tin ')

In [ ]:
print((join_list.columns))

Tính sự tương quan giữa các tin dựa trên ngành nghề, Tỉnh thành tuyển dụng, Kinh nghiệm, Mức lương

In [ ]:
df = df.drop(df.loc[df['Tiêu đề tin '].isnull()].index)
classes = df['Ngành nghề '].value_counts().sort_values(ascending=False)
keys = classes.keys().to_list()
df = df[df['Ngành nghề '].isin(keys)]
df['Ngành nghề '] = df['Ngành nghề '].apply(lambda s:s.split(','))
province = df['Tỉnh thành tuyển dụng ']
salary = df['Mức lương ']
exper = df['Kinh nghiệm ']
len(df['Ngành nghề '])

from sklearn.preprocessing import MultiLabelBinarizer,LabelEncoder
mlb = MultiLabelBinarizer()
X=mlb.fit_transform(df['Ngành nghề '])
province = mlb.fit_transform(province.values)
salary = mlb.fit_transform(salary.values)
exper = mlb.fit_transform(exper.values)
#print(X)

from sklearn.metrics.pairwise import cosine_similarity
carrier_sim = cosine_similarity(X)
province_sim = cosine_similarity(province)
salary_sim = cosine_similarity(salary)
exper_sim = cosine_similarity(exper)

carrier_sim.shape


Đánh trọng số dựa theo mức độ quan trọng của các thuộc tính

In [ ]:
weighted_sim = 0.6*carrier_sim + 0.2*salary_sim + 0.1*exper_sim + 0.1*province_sim


Tin được gợi ý sẽ là tin được những người cùng xem tin hiện tại đã xem và có độ tương quan gần nhất với tin hiện tại.

Nếu tin hiện tại có ít lượt xem, sẽ gợi ý theo những tin có cùng thuộc tính.

In [ ]:
id = current_news_id
job_index = join_list[join_list['Id tin ']==id].index.tolist()[0]
#print(job_index)
similar_job = list(enumerate(weighted_sim[job_index]))
sorted_list = sorted(similar_job,key=lambda x:x[1], reverse=True)
current_job = join_list['Tiêu đề tin '][job_index]
print("Current job: {}{}",current_job, )
print("\n-------------------------\nOthers users also see : \n")
recommend_jobs = []
recommend_jobs_id = []
for job in sorted_list:
    try:
        id = join_list['Id tin '][job[0]]
        if (id in (news_list)):
            jb = join_list['Tiêu đề tin '][job[0]]
            if jb not in recommend_jobs and jb != current_job :
                recommend_jobs.append(jb)
                recommend_jobs_id.append(int(id))
    except:
        continue
for id,job in zip(recommend_jobs_id[:11],recommend_jobs[:11]):
    print(id,job)

    
job_idx = df[df['Id tin ']==id].index.tolist()[0]
sim_job = list(enumerate(weighted_sim[job_idx]))
sorted_list_full = sorted(sim_job,key=lambda x:x[1], reverse=True)
    
    
if (len(recommend_jobs)<10):    
    print('\n--------------------------\nJobs you may concern about: \n')
    alternative = []
    alternative_id = []
    for job in sorted_list_full:
        id = df['Id tin '][job[0]]
        if (id not in (news_list)):
            alternative.append(df['Tiêu đề tin '][job[0]]) 
            alternative_id.append(int(id))
        if (len(alternative)>(10-len(recommend_jobs))):
            break;
    for job_id,job in zip(alternative_id,alternative):
        print(job_id,job)